In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile


dataset_label = "AmazonMusicalInstruments"
extract_to = "/content/data/"
root_path = "/content/drive/My Drive/Colab Data/M.Tech. Project/datasets/Amazon/"
data_path = root_path + dataset_label + "/" + dataset_label + ".zip"
tsv_path = root_path + dataset_label + "/" + dataset_label + ".tsv"
save_model_path = root_path + dataset_label + "/" + "PPLM-SavedModel"

with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

df = pd.read_json("/content/data/" + dataset_label + ".json", lines=True)
df = df[['overall', 'reviewText']]
columns = ['class', 'text']
df.columns = columns
df.drop(df[df['class'] == 3].index, inplace=True)
df['class'] = np.where(df['class'] > 3, 1, 0)
df = df.sample(frac=1)
df.reset_index(drop=True, inplace=True)
drop_quantity = df[df['class'] == 0].shape[0]
df = df.drop(df[df['class'] == 1].index[drop_quantity:])
df.reset_index(drop=True, inplace=True)

df = df.sample(frac=1)
df.reset_index(drop=True, inplace=True)
df.to_csv(tsv_path, sep='\t', index=False, header=False)

In [ ]:
%cd /content/
!git clone https://github.com/deepeshhada/PPLM
%cd /content/PPLM/
!pip install -r requirements.txt
from transformers.modeling_gpt2 import GPT2LMHeadModel
_ = GPT2LMHeadModel.from_pretrained("gpt2-medium")

/content
Cloning into 'PPLM'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 260 (delta 4), reused 7 (delta 2), pack-reused 250
Receiving objects: 100% (260/260), 2.43 MiB | 2.33 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/PPLM
     |████████████████████████████████| 1.1MB 4.0MB/s 
     |████████████████████████████████| 1.1MB 28.7MB/s 
     |████████████████████████████████| 890kB 40.2MB/s 
     |████████████████████████████████| 3.0MB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=25b4bb689b00e0e735aab16e2b6217562a3afebc4f82ebd437046cb99b5fb13a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!python run_pplm_discrim_train.py --dataset="generic" --dataset_fp="$tsv_path" --output_fp="$save_model_path" --epochs=15 --save_model --pretrained_model="gpt2" 

In [ ]:
discrim_weights_path = save_model_path + "/generic_classifier_head_epoch_15.pt"
discrim_meta_path = save_model_path + "/generic_classifier_head_meta.json"
# cond_text = "I don't think I like this show very much. However, the characters have played their part well. The flaw lies in the storyline and you can't really blame the actors."
cond_text = "This guitar is amazing but has a few problems. The tuning pegs are way too tight, and so, there's always a fear of the bridge getting snapped."

!python run_pplm.py \
--length=100 \
--discrim="generic" \
--discrim_weights="$discrim_weights_path" \
--discrim_meta="$discrim_meta_path" \
--class_label="0" \
--cond_text="$cond_text" \
--num_iterations=5 \
--stepsize=0.2 \
--temperature=1 \
--sample \
--num_samples=5 \
--gamma=1 \
--gm_scale=0.875 \
--kl_scale=0.01 \
--verbosity='quiet'

In [ ]:
discrim_weights_path = save_model_path + "/generic_classifier_head_epoch_5.pt"
discrim_meta_path = save_model_path + "/generic_classifier_head_meta.json"

!python run_pplm.py \
--length=50 \
--discrim="generic" \
--discrim_weights="$discrim_weights_path" \
--discrim_meta="$discrim_meta_path" \
--class_label="0" \
--cond_text="I don't think I like this show" \
--num_iterations=5 \
--stepsize=0.2 \
--sample \
--num_samples=5 \
--gamma=1 \
--gm_scale=0.9 \
--kl_scale=0.01 \
--verbosity='quiet'

In [ ]:
from run_pplm import run_pplm_example

run_pplm_example(
    cond_text="I think that this show is ",
    num_samples=5,
    bag_of_words='../show1.txt',
    length=50,
    stepsize=0.01,
    sample=True,
    num_iterations=5,
    window_length=5,
    gamma=1,
    gm_scale=0.875,
    kl_scale=0.01,
    verbosity='quiet'
)


= Prefix of sentence =
<|endoftext|>I think that this show is 



/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:728: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
<|endoftext|>I think that this show is  great. I think that it is so much fun to watch and I think that it will probably continue to be enjoyed.  I think that there are several reasons for that.  The first is that the writing of these characters is great

= Perturbed generated text 1 =
<|endoftext|>I think that this show is  so much fun that you should try it yourself.  This show isn't for those just getting into anime.  It's for those who have experienced the best of anime and want to be able to say "I have experienced  

= Perturbed generated text 2 =
<|endoftext|>I think that this show is  more than a bit overrated.  Not only are the characters not interesting and I don't know why there were no flashbacks, but the story is not as well thought out as the show could have been.  In the first season

= Perturbed generated text 3 =
<|endoftext|>I think that this show is  about the fact that a lot of people don't know how to deal with emotions, especially wh

In [ ]:
from run_pplm import run_pplm_example

run_pplm_example(
    cond_text="I think that this show is ",
    num_samples=5,
    bag_of_words='../show2.txt',
    length=30,
    stepsize=0.04,
    sample=True,
    num_iterations=5,
    window_length=5,
    gamma=1.5,
    gm_scale=0.875,
    kl_scale=0.01,
    verbosity='quiet'
)


= Prefix of sentence =
<|endoftext|>I think that this show is 



/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:728: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
<|endoftext|>I think that this show is  great. I think that it is so much fun to watch and I think that it will probably continue to be enjoyed.  I think that

= Perturbed generated text 1 =
<|endoftext|>I think that this show is  the most  interesting  show  of all the movies in that it is not just a movie but the story that is told through the film

= Perturbed generated text 2 =
<|endoftext|>I think that this show is  a little more than an experiment.  I like it a bit more experimental in its style and tone than mystery mystery, mystery mystery mystery mystery mystery

= Perturbed generated text 3 =
<|endoftext|>I think that this show is  so much different than most anime I watch. It is not a comedy show, but a dark mystery mystery mystery. It has mystery and mystery mystery

= Perturbed generated text 4 =
<|endoftext|>I think that this show is ____ because of that one character. He's just a bad, sad, annoying guy. The show is mystery mystery drama com

In [ ]:
from run_pplm import run_pplm_example

run_pplm_example(
    cond_text="I don't think I like this show",
    num_samples=5,
    bag_of_words='../Enya_neg.txt',
    length=50,
    stepsize=0.03,
    sample=True,
    num_iterations=5,
    window_length=5,
    gamma=1.5,
    gm_scale=0.95,
    kl_scale=0.01,
    verbosity='quiet'
)

In [ ]:
from run_pplm import run_pplm_example

run_pplm_example(
    cond_text="I don't think I like this show very much. However, the characters have played their part well. The flaw lies in the storyline and you can't really blame the actors.",
    num_samples=5,
    discrim='sentiment',
    class_label='very_negative',
    length=70,
    stepsize=0.2,
    sample=True,
    num_iterations=10,
    window_length=5,
    gamma=1,
    gm_scale=0.875,
    kl_scale=0.01,
    verbosity='quiet'
)

= Prefix of sentence =
<|endoftext|>I don't think I like this show very much. However, the characters have played their part well. The flaw lies in the storyline and you can't really blame the actors.



/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:712: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
<|endoftext|>I don't think I like this show very much. However, the characters have played their part well. The flaw lies in the storyline and you can't really blame the actors. The show isn't very good.

I think the main problem is not only the plot but the characters and how they relate to each other. Some of the characters don't seem to have any depth. Some of them, they are so annoying or even too annoying that they are annoying at times. They have nothing to do with each other.

= Perturbed generated text 1 =
<|endoftext|>I don't think I like this show very much. However, the characters have played their part well. The flaw lies in the storyline and you can't really blame the actors. The show has a tendency towards horrible plot-holes and bad character designs, especially for a comedy show aimed at teenagers. It is not a good show. The plotline of the show is so terrible that you just have to watch it and find it annoying because of that. The writing